In [1]:
# Useful starting lines
%matplotlib inline
import datetime
import numpy as np
import matplotlib.pyplot as plt
%load_ext autoreload
%autoreload 2

## Load the training data into feature matrix, class labels, and event ids:

In [2]:
from proj1_helpers import *
DATA_TRAIN_PATH = '../data/train.csv'
y, x, ids = load_csv_data(DATA_TRAIN_PATH)

def standardize(x):
    """Standardize the original data set."""
    mean_x = np.mean(x)
    x = x - mean_x
    std_x = np.std(x)
    x = x / std_x
    return x, mean_x, std_x

x, mean_x, std_x = standardize(x)
tX = np.c_[np.ones(y.shape[0]), x]

## Do your thing crazy machine learning thing here :) ...

In [3]:
# ###############################################
#
# --- Helper functions for Linear Regression
#
# ###############################################

def compute_loss(y, tx, w):
    """Calculate the MSE."""
    e = y.reshape(-1,1) - (tx @ w)
    return np.square(e).mean()/2


def compute_gradient(y, tx, w):
    """Compute the gradient."""
    e = y.reshape(-1,1) - (tx @ w)
    gradL = (-1/tx.shape[0]) * (tx.T @ e)
    return gradL

In [4]:
def least_squares_GD(y, tx, initial_w, max_iters, gamma):
    """Gradient descent algorithm."""
    # Define parameters to store w and loss
    ws = [initial_w]
    losses = []
    w = initial_w
    for n_iter in range(max_iters):
        print(n_iter)
        grad = compute_gradient(y, tx, w)
        loss = compute_loss(y, tx, w)
        w = w - gamma * grad
        
        # store w and loss
        ws.append(w)
        losses.append(loss)
        print("Gradient Descent({bi}/{ti}): loss={l}, w0={w0}, w1={w1}".format(
              bi=n_iter, ti=max_iters - 1, l=loss, w0=w[0], w1=w[1]))

    return losses, ws

def batch_iter(y, tx, batch_size, num_batches=1, shuffle=True):
    """
    Generate a minibatch iterator for a dataset.
    Takes as input two iterables (here the output desired values 'y' and the input data 'tx')
    Outputs an iterator which gives mini-batches of `batch_size` matching elements from `y` and `tx`.
    Data can be randomly shuffled to avoid ordering in the original data messing with the randomness of the minibatches.
    Example of use :
    for minibatch_y, minibatch_tx in batch_iter(y, tx, 32):
        <DO-SOMETHING>
    """
    data_size = len(y)

    if shuffle:
        shuffle_indices = np.random.permutation(np.arange(data_size))
        shuffled_y = y[shuffle_indices]
        shuffled_tx = tx[shuffle_indices]
    else:
        shuffled_y = y
        shuffled_tx = tx
    for batch_num in range(num_batches):
        start_index = batch_num * batch_size
        end_index = min((batch_num + 1) * batch_size, data_size)
        if start_index != end_index:
            yield shuffled_y[start_index:end_index], shuffled_tx[start_index:end_index]

def least_squares_SGD(y, tx, initial_w, batch_size, max_iters, gamma):
    """Stochastic gradient descent algorithm."""
    # Define parameters to store w and loss
    ws = [initial_w]
    losses = []
    w = initial_w
    num_batches = 100#int(len(y) / batch_size)
    for n_iter in range(max_iters):
        for sy, stx in batch_iter(y, tx, batch_size, int(tx.shape[0]/batch_size)):
            #w = w.reshape(-1, 1)
            grad = compute_gradient(sy, stx, w)
            loss = compute_loss(sy, stx, w)
            w = w - gamma * grad
            # store w and loss
            ws.append(w)
            losses.append(loss)
        print("Gradient Descent({bi}/{ti}): loss={l}, w0={w0}, w1={w1}".format(
            bi=n_iter, ti=max_iters - 1, l=loss, w0=w[0], w1=w[1]))

    return losses, ws

In [5]:
# Linear regression parameters initialization
max_iters = 50
gamma = 10**(-4)
initial_w = np.zeros((tX.shape[1], 1))

#least_squares_GD(y, tX, initial_w, max_iters, gamma)

In [6]:
# Define the parameters of the algorithm.
max_iters = 50
gamma = 0.05
batch_size = int(tX.shape[0]/1000)

# Initialization
w_initial = np.zeros((tX.shape[1], 1))

# Start SGD.
start_time = datetime.datetime.now()
sgd_losses, sgd_ws = least_squares_SGD(y, tX, w_initial, batch_size, max_iters, gamma)
end_time = datetime.datetime.now()

# Print result
exection_time = (end_time - start_time).total_seconds()
print("SGD: execution time={t:.3f} seconds".format(t=exection_time))

Gradient Descent(0/49): loss=0.38184870542538907, w0=[-0.05770223], w1=[0.19270056]
Gradient Descent(1/49): loss=0.36634551072041804, w0=[-0.00322107], w1=[0.17178309]
Gradient Descent(2/49): loss=0.36222469101211685, w0=[0.03475616], w1=[0.13885514]
Gradient Descent(3/49): loss=0.34527446649702653, w0=[0.06677186], w1=[0.13969609]
Gradient Descent(4/49): loss=0.32274909794268153, w0=[0.08149915], w1=[0.12756728]
Gradient Descent(5/49): loss=0.3438473473074282, w0=[0.09741957], w1=[0.12898565]
Gradient Descent(6/49): loss=0.3409438100945276, w0=[0.10362607], w1=[0.13373293]
Gradient Descent(7/49): loss=0.4039193118013616, w0=[0.11000043], w1=[0.11041358]
Gradient Descent(8/49): loss=0.37797444151654325, w0=[0.10996818], w1=[0.10146787]
Gradient Descent(9/49): loss=0.3548202544311354, w0=[0.10226579], w1=[0.09653837]
Gradient Descent(10/49): loss=0.35500448414469543, w0=[0.10561007], w1=[0.12353253]
Gradient Descent(11/49): loss=0.3826763843939488, w0=[0.10279432], w1=[0.09982442]
Gradi

In [16]:
DATA_TESTPATH = '../data/test.csv'
_, tX_test, ids_test = load_csv_data(DATA_TEST_PATH)
tX_test, mean_x_test, std_x_test = standardize(tX_test)
tXtest = np.c_[np.ones(tX_test.shape[0]), tX_test]

OUTPUT_PATH = '../data/submission.csv'
y_pred = predict_labels(sgd_ws[-1], tX_test)
create_csv_submission(ids_test, y_pred, OUTPUT_PATH)

ValueError: shapes (568238,30) and (31,1) not aligned: 30 (dim 1) != 31 (dim 0)

In [7]:
def least_squares(y, tx):
    """calculate the least squares."""
    a = tx.T.dot(tx)
    b = tx.T.dot(y)
    return np.linalg.solve(a, b)


def ridge_regression(y, tx, lambda_):
    """implement ridge regression."""
    m, n = tx.shape[0], tx.shape[1]
    lambda_prime = lambda_ * 2 * m * np.eye(n)
    a = tx.T.dot(tx) + lambda_prime
    b = tx.T.dot(y) 
    return np.linalg.solve(a, b)